In [1]:
import pandas as pd
from itertools import product
from tqdm.notebook import tqdm

import sys, os

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2
from modules.config import *
from modules.svm import *

In [2]:
all_possible_metas = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100],  'gamma': [-1],                          'degree': [-1],         'max_iter': [1000000]},
    {'kernel': ['rbf'],    'C': [0.1, 1, 10, 100],  'gamma': [0.1, 0.01, 0.001, 0.0001],    'degree': [-1],         'max_iter': [1000000]},
    {'kernel': ['poly'],   'C': [1, 10, 100, 1000], 'gamma': [-1],                          'degree': [2, 3, 4, 5], 'max_iter': [1000000]}
]

In [3]:
execute_stage(
    'first_stage',
    SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
    SVM_SECOND_STAGE_DEMAND_RESULTS_PATH,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
    all_possible_metas,
    get_demand_model_data,
)

0it [00:00, ?it/s]

In [4]:
results = pd.read_parquet(SVM_FIRST_STAGE_DEMAND_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False).head(2)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,param_max_iter,params,...,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score,n_iter,h3_res,time_interval_length,param_gamma,param_degree
29,2,4995,23.300784,0.309178,4.868410,0.110677,1.0,rbf,1000000,"{'C': 1.0, 'degree': None, 'gamma': 0.1, 'kern...",...,-0.831889,-0.716120,-0.765514,-0.784402,0.040580,0,8,6,0.100,NaN
28,2,4995,20.118273,0.170544,4.109246,0.167242,1.0,rbf,1000000,"{'C': 1.0, 'degree': None, 'gamma': 0.01, 'ker...",...,-1.317467,-1.166759,-1.221405,-1.253317,0.053110,0,8,6,0.010,NaN
27,1,1665,4.062596,0.084986,0.845498,0.077482,1.0,rbf,1000000,"{'C': 1.0, 'degree': None, 'gamma': 0.1, 'kern...",...,-0.831517,-0.888092,-1.171911,-0.805592,0.228088,0,8,6,0.100,NaN
26,1,1665,3.630293,0.187915,0.898211,0.058731,1.0,rbf,1000000,"{'C': 1.0, 'degree': None, 'gamma': 0.01, 'ker...",...,-1.296860,-1.333049,-1.637696,-1.248364,0.247997,0,8,6,0.010,NaN
23,1,1665,4.855947,0.285339,0.738693,0.069901,100.0,rbf,1000000,"{'C': 100.0, 'degree': None, 'gamma': 0.001, '...",...,-1.187792,-1.235244,-1.530168,-1.144765,0.245565,0,8,6,0.001,NaN


In [5]:
results.sort_values(by='mean_test_score', ascending=False).groupby('param_kernel')[[
    'mean_test_score',
    'params',
]].first()

param_kernel
linear   -1.646263
poly     -1.512246
rbf      -1.376150
Name: mean_test_score, dtype: float64

In [6]:
results.groupby('param_kernel').mean_fit_time.sum() / 60

param_kernel
linear    3.273515
poly      1.212989
rbf       1.178325
Name: mean_fit_time, dtype: float64

In [7]:
resolutions = list(product(PREDICTIVE_H3_RESOLUTIONS, CALC_TIME_INTERVAL_LENGTHS))
resolutions = resolutions + (ADDITIONAL_PREDICTIVE_RESOLUTIONS)
resolutions

[(7, 1), (7, 2), (7, 6), (7, 24), (8, 1), (8, 2), (8, 6), (8, 24), (9, 24)]

In [8]:
for h3_res, time_interval_length in tqdm(resolutions):
    execute_stage(
        'second_stage',
        SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
        SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
        h3_res,
        time_interval_length,
        all_possible_metas,
        get_demand_model_data,
    )

  0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
results = pd.read_parquet(SVM_SECOND_STAGE_DEMAND_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,param_max_iter,...,mean_train_score,std_train_score,n_iter,h3_res,time_interval_length,test_mse,test_rmse,test_mae,test_non_zero_mape,test_zero_accuracy
4,0,5000,11.370774,0.771222,2.659625,0.498870,100.0,0.1,rbf,1000000,...,-0.021526,0.002141,9488,8,1,0.305967,0.553143,0.372539,0.314128,0.999938
5,0,5000,10.795261,0.412445,2.391331,0.193675,100.0,0.1,rbf,1000000,...,-0.022847,0.000971,8337,8,2,0.554213,0.744455,0.473506,0.370717,0.999728
6,0,5000,9.888528,0.776858,2.390681,0.444566,100.0,0.1,rbf,1000000,...,-0.122380,0.005369,9265,8,6,1.800588,1.341860,0.783819,0.509157,0.998376
0,0,5000,4.962967,0.297516,1.872338,1.993469,100.0,0.1,rbf,1000000,...,-0.441346,0.037841,9463,7,1,5.990319,2.447513,1.483825,0.651693,0.999193
1,0,5000,4.645118,0.205902,0.903195,0.070386,100.0,0.1,rbf,1000000,...,-1.961534,0.247418,10209,7,2,17.836231,4.223296,2.367089,0.758412,0.997688
2,0,5000,4.781864,0.363426,1.084935,0.131030,100.0,0.1,rbf,1000000,...,-25.859211,1.090046,12008,7,6,97.970889,9.898024,5.209171,0.952846,0.989637
3,0,5000,4.645064,0.499779,0.899562,0.059447,100.0,0.1,rbf,1000000,...,-384.796477,20.819367,10927,7,24,636.583650,25.230609,12.633961,1.042509,0.996542


In [ ]:
results_first_stage = pd.read_parquet(SVM_FIRST_STAGE_DEMAND_RESULTS_PATH)
results_second_stage = pd.read_parquet(SVM_FIRST_STAGE_DEMAND_RESULTS_PATH)
results_first_stage.to_parquet(SVM_FIRST_STAGE_DEMAND_RESULTS_PATH + '_' + str(SVM_MAX_TRAIN_SET_SIZE))
results_second_stage.to_parquet(SVM_FIRST_STAGE_DEMAND_RESULTS_PATH + '_' + str(SVM_MAX_TRAIN_SET_SIZE))